In [1]:
! pip install -r '../requirements.txt'


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import torch
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn
import numpy as np
from src.model import UNet
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms.functional as TF
import albumentations as A
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt

/Users/lancehendricks/Documents/College Coding/ML/ISIC/.venv/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.1' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
device=torch.device('mps')

In [4]:
# load image and mask files
image_dir = '../data/images'
mask_dir = '../data/masks'

image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')])
mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.png')])
assert len(image_files) == len(mask_files), "Number of images and masks must match!"

In [5]:
image_dim = 256

# define image and mask transforms
image_transform = transforms.Compose([
    transforms.Resize((image_dim, image_dim)),
    transforms.ToTensor()
])

# resize targets to 68x68 (match model output)
def mask_transform(mask):
    mask = TF.resize(mask, size=(68, 68), interpolation=TF.InterpolationMode.NEAREST)
    mask = torch.tensor(np.array(mask), dtype=torch.long, device=device)
    mask[mask==255]=1
    return mask
    
# define augmentation pipeline for a more flexible model
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], additional_targets={'mask': 'mask'})

In [6]:
# train test split
train_images, val_images, train_masks, val_masks = train_test_split(image_files, mask_files, test_size=0.2, random_state=42)

In [7]:
# create torch dataset to easily load and preprocess images
class SegmentationDataset(Dataset):
    
    def __init__(self, image_paths, mask_paths, image_transform, mask_transform, augmentation=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        self.augmentation = augmentation

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # load image and mask
        image = Image.open(self.image_paths[idx]).convert("RGB")
        mask = Image.open(self.mask_paths[idx]).convert("L")  # Grayscale
        
        if self.augmentation:
            # image and mask must be numpy arrays for augmentation
            image = np.array(image)
            mask = np.array(mask)
            augmented = self.augmentation(image=image, mask=mask)
            image, mask = augmented['image'], augmented['mask']
            # convert image and mask back to PIL for additional transformations
            image = Image.fromarray(image)
            mask = Image.fromarray(mask)
            
        image = self.image_transform(image)
        mask = self.mask_transform(mask)
        

        return image, mask

In [8]:
# define train and val datasets and dataloaders
train_dataset = SegmentationDataset(train_images, train_masks, image_transform=image_transform, mask_transform=mask_transform, augmentation=augmentation)
val_dataset = SegmentationDataset(val_images, val_masks, image_transform=image_transform, mask_transform=mask_transform, augmentation=None) # don't touch validation images

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

print('size of training data', len(train_dataset))
print('size of validation data', len(val_dataset))

# check data
for images, masks in train_loader:
    print(f"Image batch shape: {images.shape}")
    print(f"Mask batch shape: {masks.shape}")
    break

size of training data 2075
size of validation data 519
Image batch shape: torch.Size([8, 3, 256, 256])
Mask batch shape: torch.Size([8, 68, 68])


In [9]:
# finally start training the model

num_classes = 2

In [10]:
# get checkpoint to save model
#checkpoint = torch.load('../checkpoints/checkpoint.pth')

In [11]:
# init loss with class weights
def calculate_class_weights(data_loader, num_classes):
    pixel_counts = torch.zeros(num_classes)  # To store counts for each class

    # Loop through the dataset
    for _, masks in data_loader:
        # Count pixels per class
        for c in range(num_classes):
            pixel_counts[c] += masks[:,c,:,:].sum()
        
        print(pixel_counts)

    # Total pixels in the dataset
    total_pixels = pixel_counts.sum()

    # Inverse frequency
    class_weights = total_pixels / (num_classes * pixel_counts)
    return class_weights

#class_weights = calculate_class_weights(train_loader, num_classes)
class_weights = torch.tensor([0.6341, 2.3639])

In [12]:
# init model
unet = UNet(num_classes, image_dim)
#unet.load_state_dict(checkpoint['state_dict'])
unet = unet.to(device)

AttributeError: cannot assign module before Module.__init__() call

In [13]:
# create a custom dice loss class
class DiceLoss(nn.Module):
    
    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 1.
        
    def forward(self, preds, targets):
        preds = preds.softmax(dim=1)[:, 1, :, :]# [batch_size, height, width]
        preds_flat = preds.view(preds.size(0),-1)
        target_flat = targets.view(targets.size(0),-1)
        intersection = (preds_flat * target_flat).sum(dim=1)
        dice = (2. * intersection + self.smooth) / (preds_flat.sum(dim=1) + target_flat.sum(dim=1) + self.smooth)
        return 1 - dice.mean()

# create hybrid dice and bce loss to optimize for both
class HybridDiceBCELoss(nn.Module):
    def __init__(self, weights=None):
        super(HybridDiceBCELoss, self).__init__()
        self.dice_loss = DiceLoss()
        self.bce_loss = nn.CrossEntropyLoss(weight=weights)
        
    def forward(self, preds, targets):
        dice = self.dice_loss(preds, targets)
        bce_loss = self.bce_loss(preds, targets)
        return dice + bce_loss
    
criterion = HybridDiceBCELoss(class_weights)

In [14]:
# init optimizer
optimizer = torch.optim.Adam(unet.parameters(), lr=1e-4)
#optimizer.load_state_dict(checkpoint['optimizer'])

In [15]:
# add scheduler to prevent overfitting
scheduler = ReduceLROnPlateau(optimizer,
                              mode='min',
                              factor=0.1,
                              patience=3,
                              threshold=1e-3,
                              verbose=True)

/Users/lancehendricks/Documents/College Coding/ML/ISIC/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [16]:
# use to calculate mean intersection over union for evaluating performance (with one hot encodings)
def get_miou(predictions, targets):
    # confusion matrix between predicted and ground truth class labels
    conf_matrix = torch.zeros(num_classes, num_classes, device=device)

    predictions = predictions.flatten()
    targets = targets.flatten()

    #print('pred',predictions)
    #print('targets',targets)

    # rows are predictions, cols are targets
    for c in range(num_classes):
        for t in range(num_classes):
            conf_matrix[c,t] += torch.sum((predictions == c) & (targets == t))

    #print(conf_matrix)

    # get iou for each, then calculate mean later
    class_ious = []
    for c in range(num_classes):
        tp = conf_matrix[c,c]
        fp = conf_matrix[c,:].sum() - conf_matrix[c,c]
        fn = conf_matrix[:,c].sum() - conf_matrix[c,c]

        U = tp + fp + fn
        #print(U)

        if U == 0: # prevent division by 0
            class_ious.append(float('nan'))
        else:
            class_ious.append(tp.float()/U)

    #print(class_ious)

    return torch.nanmean(torch.tensor(class_ious, device=device))

In [17]:
# use to evaluate with val data
def eval_model(model, val_loader, criterion):
    model.eval()
    loss_sum = 0
    miou_sum = 0
    batches = len(val_loader)
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)

            out = model(images)
            loss = criterion(out.to('cpu'), masks.to('cpu'))
            loss_sum += loss.item()

            predictions = torch.argmax(out, dim=1)
            miou = get_miou(predictions, masks.to(device)).item()
            miou_sum += miou

    avg_val_loss = loss_sum / batches
    avg_val_miou = miou_sum / batches
    return avg_val_loss, avg_val_miou

In [18]:
# init summary writer
writer = SummaryWriter('../runs/unet')

In [19]:
# training loop

cur_epoch = 0
#cur_epoch = checkpoint['epoch']

early_stop_tolerance = 3
best_val_loss = float('inf')
epochs_wo_improve = 0

epochs = 50
unet.train()
for epoch in range(cur_epoch, cur_epoch + epochs):
    mious = 0
    losses = 0
    n = len(train_loader)
    for batch_idx, (images, masks)in enumerate(train_loader):
        #print(images.shape, masks.shape)
        #print(torch.unique(masks))
        images = images.to(device)
        masks = masks.to(device)
        out = unet(images)
        #out = out.to(device)
        #print(out.shape)
        #print(masks.shape)
        #print(f"Output device: {out.dtype}")
        #print(f"Mask device: {masks.dtype}")
        # have to put temporarily on cpu as torch has some sort of issue with mps here :(
        loss = criterion(out.to('cpu'), masks.to('cpu'))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # turn probabilities into hard predictions
        predictions = torch.argmax(out, dim=1)

        print('loss',loss.item())
        #print('pred',torch.unique(predictions))
        #print('mask',torch.unique(masks))
        #print('out',torch.unique(out))
        miou = get_miou(predictions, masks).item()
        print('miou',miou)

        losses += loss.item()
        mious += miou

        # add train and mIoU for individual
        writer.add_scalar('Loss/train', loss.item(), epoch * n + batch_idx)
        writer.add_scalar('mIoU/train', miou, epoch * n + batch_idx)

    # add train loss and mIoU for batch
    print("epoch mIoU:",mious/n)
    print("epoch loss:",losses/n)
    writer.add_scalar(' train loss/epoch', losses/n, epoch)
    writer.add_scalar('train mIoU/epoch', mious/n, epoch)

    # add val loss and mIoU
    val_loss, val_miou = eval_model(unet, val_loader, criterion)
    unet.train()
    writer.add_scalar('val loss/epoch', val_loss, epoch)
    writer.add_scalar('val mIoU/epoch', val_miou, epoch)
    #add images at end of each epoch to visualize predictions
    sample_images = images[:5]
    sample_preds = predictions[:5]
    sample_masks = masks[:5]
    #print(sample_images.shape, sample_masks.shape, sample_preds.shape)

    writer.add_images("Images", sample_images, epoch)
    writer.add_images("Predictions", sample_preds.unsqueeze(1), epoch)
    writer.add_images("Masks", sample_masks.unsqueeze(1), epoch)

    # save progress
    checkpoint = {
        'epoch': epoch + 1,
        'state_dict': unet.state_dict(),
        'optimizer': optimizer.state_dict(),
    }

    torch.save(checkpoint, '../checkpoints/checkpoint.pth')
    scheduler.step(val_loss)

    if val_loss < best_val_loss - 1e-4:
        best_val_loss = val_loss
        epochs_wo_improve = 0
    else:
        epochs_wo_improve += 1

    if epochs_wo_improve >= early_stop_tolerance:
        print(f"Early stopping at epoch {epoch+1}")
        break

loss 0.6049095392227173
miou 0.7346100807189941
loss 0.38344937562942505
miou 0.8206238746643066
loss 0.22905069589614868
miou 0.9189150333404541
loss 0.23525968194007874
miou 0.895673394203186
loss 0.44414860010147095
miou 0.8251659870147705
loss 0.22142654657363892
miou 0.8665119409561157
loss 0.24440717697143555
miou 0.871006965637207
loss 0.4302530288696289
miou 0.8160942792892456
loss 0.2563096582889557
miou 0.8945494890213013
loss 0.5308332443237305
miou 0.7787142395973206
loss 0.5265092849731445
miou 0.8015982508659363
loss 0.4301712214946747
miou 0.7048381567001343
loss 0.3067742586135864
miou 0.8649636507034302
loss 0.3267757296562195
miou 0.8868972063064575
loss 0.37318944931030273
miou 0.8010859489440918
loss 0.49079519510269165
miou 0.7973199486732483
loss 0.30000555515289307
miou 0.8389931917190552
loss 0.28726059198379517
miou 0.8537798523902893
loss 0.4400833249092102
miou 0.790977954864502
loss 0.41643545031547546
miou 0.8565963506698608
loss 0.28314000368118286
miou 0.

In [20]:
# close writer
writer.close()

In [ ]:
def visualize_predictions(inputs, predictions, ground_truths, n_samples=5):
    """
    Visualize input images, predicted masks, and ground truth masks.

    Parameters:
    - inputs: Batch of input images (Tensor or NumPy array).
    - predictions: Batch of predicted masks (Tensor or NumPy array).
    - ground_truths: Batch of ground truth masks (Tensor or NumPy array).
    - n_samples: Number of samples to visualize.
    """
    n_samples = min(n_samples, len(inputs))
    plt.figure(figsize=(15, 5 * n_samples))

    for i in range(n_samples):
        plt.subplot(n_samples, 3, i * 3 + 1)
        plt.imshow(inputs[i][0], cmap='gray')
        plt.title('Input Image')
        plt.axis('off')

        plt.subplot(n_samples, 3, i * 3 + 2)
        plt.imshow(predictions[i][0], cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')

        plt.subplot(n_samples, 3, i * 3 + 3)
        plt.imshow(ground_truths[i][0], cmap='gray')
        plt.title('Ground Truth Mask')
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
unet.eval()
for i in range(3):
    image, mask = val_dataset.__getitem__(i)
    
    pred = unet(image)

    plt.imshow(image[0], cmap='gray')
    plt.title('Input Image')
    plt.axis('off')

    plt.imshow(pred[0], cmap='gray')
    plt.title('Predicted Mask')
    plt.axis('off')

    plt.imshow(mask[0], cmap='gray')
    plt.title('Ground Truth Mask')
    plt.axis('off')